## Creating "virtual H&E" for ISS samples from gene expression

In [19]:
import scanpy as sc
import pandas as pd
import cv2
from PIL import Image as Im
import numpy as np
import matplotlib.pyplot as plt

In [20]:
data_dir = '/nfs/team292/vl6/FetalReproductiveTract/ISS/'
sample = 'CM009'

In [21]:
adata_sp = sc.read(data_dir + 'RV_RPT_CM009_n_gene_min_4.h5ad')

In [22]:
adata_sp

AnnData object with n_obs × n_vars = 56627 × 171
    obs: 'sample', 'total_counts', 'n_genes_by_counts', 'leiden'
    var: 'total_counts', 'n_cells_by_counts'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [23]:
import sys
utils_dir = '/nfs/team292/vl6/RepTract/'
sys.path.append(utils_dir)

import reptract_utils

In [24]:
# Normalise spatial data 
adata_sp.raw = adata_sp.copy()
adata_sp = reptract_utils.normalize_log_transform(adata_sp) ### this is my own wrapped function for normalisation and log transformation in scanpy 

In [25]:
# RED
geneind1 = np.where(adata_sp.var_names == 'EPCAM')[0]
genedata1 = np.squeeze(np.asarray(adata_sp.X[:,geneind1]))
norm_factor1 = 255/np.max(genedata1)
print(norm_factor1)

29.311419


In [26]:
# GREEN
geneind2 = np.where(adata_sp.var_names == 'PDGFRA')[0]
genedata2 = np.squeeze(np.asarray(adata_sp.X[:,geneind2]))
norm_factor2 = 255/np.max(genedata2)
print(norm_factor2)

27.685972


In [27]:
# BLUE
geneind3 = np.where(adata_sp.var_names == 'MYH11')[0]
genedata3 = np.squeeze(np.asarray(adata_sp.X[:,geneind3]))
norm_factor3 = 255/np.max(genedata3)
print(norm_factor3)

28.373302


In [28]:
# Get spatial data from ISS object
point_cloud = adata_sp.obsm['spatial']

In [29]:
# Determine the XY range of the point cloud data
x_min, y_min = np.min(point_cloud, axis=0)
x_max, y_max = np.max(point_cloud, axis=0)
print(x_min, y_min, x_max, y_max)

2323.8596388339074 1964.4552241207325 40968.72110587392 32611.580577269997


In [30]:
# Calculate the width and height of the image
width = int(x_max)
height = int(y_max)
print('width: {}, height: {}'.format(width, height))

width: 40968, height: 32611


In [31]:
# Create an empty image of the same size
image = np.zeros((height, width,3), dtype=np.uint8)  # RGB image
image.shape

(32611, 40968, 3)

In [32]:
# Map points to pixels and assign color values
for ind,point in enumerate(point_cloud):
    x, y = point
    pixel_x = int(x)
    pixel_y = int(y)
    image[pixel_y-1, pixel_x-1] = (norm_factor1*genedata1[ind],norm_factor2*genedata2[ind],norm_factor3*genedata3[ind])  # Assign color to the pixel

In [33]:
#Split the image into the B,G,R components.
b,g,r = cv2.split(image)
kernel = np.ones((25, 25), np.uint8)

In [34]:
# Perform dilation on each color channel.
b_dilated = cv2.dilate(b, kernel, iterations = 2)
g_dilated = cv2.dilate(g, kernel, iterations = 2)
r_dilated = cv2.dilate(r, kernel, iterations = 2)

In [35]:
# Merge the channels back together.
dilated_img = np.dstack((b_dilated, g_dilated, r_dilated,np.ones((height, width), dtype=np.uint8)*255))  # RGBA image
ImP = Im.fromarray(dilated_img)

In [37]:
ImP.save('/nfs/team292/vl6/FetalReproductiveTract/ISS/CM009/CM009.png')